In [1]:
from datetime import datetime
import pandas as pd

In [2]:
# Load dataset
path = '/kaggle/input/open-shopee-code-league-logistic/delivery_orders_march.csv'
dataset = pd.read_csv(path)

In [3]:
# Show sample data
dataset.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,1583138397,1.583385e+09,NaN,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,1583309968,1.583463e+09,1.583799e+09,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,1583306434,1.583460e+09,NaN,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."
3,2221066352,1583419016,1.583556e+09,NaN,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number..."
4,2222478803,1583318305,1.583480e+09,NaN,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...


In [4]:
# Show column type
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3176313 entries, 0 to 3176312
Data columns (total 6 columns):
 #   Column               Dtype  
---  ------               -----  
 0   orderid              int64  
 1   pick                 int64  
 2   1st_deliver_attempt  float64
 3   2nd_deliver_attempt  float64
 4   buyeraddress         object 
 5   selleraddress        object 
dtypes: float64(2), int64(2), object(2)
memory usage: 145.4+ MB


# Data Preprocessing

In [5]:
# Change column type of orderid to string
dataset['orderid'] = dataset['orderid'].astype(str)

In [6]:
# Change the format of date columns
for column in ['pick', '1st_deliver_attempt', '2nd_deliver_attempt']:
    dataset[column] = dataset[column].apply(
        lambda cell : cell if pd.isna(cell) else datetime.fromtimestamp(cell)
    )
    dataset[column] = dataset[column].dt.date

# Data Analysis

In [7]:
# Get the origin and destination
for column in ['buyeraddress', 'selleraddress']:
    dataset[column] = dataset[column].apply(
        lambda cell : cell.lower().split(' ')[-1]
    )

In [8]:
# Function to get the number of holidays
def get_holidays(start, end):
    
    # Initialization
    holidays = 0
    
    # Get the number of holidays
    if pd.notnull(end):
        
        # Get dates in between
        dates = pd.date_range(start, end)

        # Get the number of sundays
        sundays = dates.weekday.isin([6]).sum()
        holidays = holidays + sundays

        # Get the number of public holidays
        public_holidays = ['2020-03-25', '2020-03-30', '2020-03-31']
        holidays = holidays + dates.isin(public_holidays).sum()
    
    # Get the total number of holidays
    return(holidays)

In [9]:
# Get the number of holidays before the first attempt
print(datetime.now())
dataset['holiday_1'] = dataset.apply(
    lambda row: get_holidays(row['pick'], row['1st_deliver_attempt']), 
    axis=1
)
print(datetime.now())

2020-07-11 13:34:15.647981
2020-07-11 14:21:17.797207


In [10]:
# Get the number of holidays before the second attempt
print(datetime.now())
dataset['holiday_2'] = dataset.apply(
    lambda row: get_holidays(row['1st_deliver_attempt'], row['2nd_deliver_attempt']), 
    axis=1
)
print(datetime.now())

2020-07-11 14:21:17.808861
2020-07-11 14:43:28.950185


In [11]:
# Function to get the delivery time
def get_time(pick, delivered, holidays):
    
    # Initialization
    delivery_time = 0
    
    # Get delivery time
    if pd.notnull(delivered):
        delivery_time = (delivered - pick).days
        delivery_time = delivery_time - holidays
    
    return(delivery_time)

In [12]:
# Get delivery time for the first attempt
dataset['delivery_1'] = dataset.apply(
    lambda row: get_time(
        row['pick'], 
        row['1st_deliver_attempt'], 
        row['holiday_1']
    ), 
    axis=1
)

In [13]:
# Get delivery time for the second attempt
dataset['delivery_2'] = dataset.apply(
    lambda row: get_time(
        row['1st_deliver_attempt'], 
        row['2nd_deliver_attempt'], 
        row['holiday_2']
    ), 
    axis=1
)

In [14]:
# Function to get SLA
def get_sla(origin, destination):
    
    if origin == 'manila' and destination == 'manila':
        sla = 3
    elif origin in ['manila', 'luzon'] and destination == 'luzon':
        sla = 5
    else:
        sla = 7
    
    return(sla)

In [15]:
# Get SLA
dataset['sla'] = dataset.apply(
    lambda row: get_sla(row['selleraddress'], row['buyeraddress']), 
    axis=1
)

In [16]:
# Function to get status
def get_status(sla, delivery_1, delivery_2):
    
    if delivery_1 > sla or delivery_2 > 3:
        late = 1
    else:
        late = 0
    
    return(late)

In [17]:
# Get delivery status
dataset['is_late'] = dataset.apply(
    lambda row: get_status(row['sla'], row['delivery_1'], row['delivery_2']),
    axis=1
)

In [18]:
# Export the result to csv
dataset.loc[:, ['orderid', 'is_late']].to_csv('submission.csv', index=False)